# QCIntegrals

In [1]:
using HypergeometricFunctions
using SpecialFunctions
using LinearAlgebra
using BenchmarkTools

In [2]:
function doublefactorial(n)
    if (n <= 1)
        return 1
    end
    return n * doublefactorial(n - 2)
end

doublefactorial (generic function with 1 method)

In [3]:
function F_n(n,x)
    if(x!=0)
        return gamma(0.5 + n) * gamma_inc(0.5 + n, x, 0)[1] / (2*x^(0.5 + n))
    else
        return HypergeometricFunctions.M(n+1/2,n+3/2,-x)/(2*n+1)
    end
end

F_n (generic function with 1 method)

## Shell

In [4]:
struct shell
    l::Int8
    exp::Vector{Float64}
    coef::Vector{Float64}
    coord::Vector{Float64}
    N::Vector{Float64}
    orientaciones::Vector{Vector{Int}}
end

function build_shell(l,exp,coef,coord;normalized=true)

    #normalizacion
    for i in eachindex(exp)
        coef[i] = coef[i]*(2*exp[i]/pi)^(3/4)*(4*exp[i])^(l/2)
    end
    
    norm = 0
    for j in eachindex(exp)
        for k in eachindex(exp)
            norm += coef[j]*coef[k]*(pi/(exp[j]+exp[k]))^(3/2)/(2*(exp[j]+exp[k]))^(l)
        end
    end

    for i in eachindex(exp)
        coef[i] = coef[i]/sqrt(norm)
    end
        
    orientaciones = []
    for lx in l:-1:0
        for ly in l-lx:-1:0
            lz = l-lx-ly
            append!(orientaciones,[[lx,ly,lz]])      
        end
    end        

    N = ones(Int((l+1)*(l+2)/2))
    if normalized
        for (i,(lx,ly,lz)) in enumerate(orientaciones)
            N[i] = sqrt(1/(doublefactorial(2*lx-1)*doublefactorial(2*ly-1)*doublefactorial(2*lz-1)))
        end
    end
    
    return shell(l, exp, coef, coord, N, orientaciones)
    
end

function build_zero_shell()

    #normalizacion
    l = 0
    exp = [0.0]
    coef = [1.0]
    coord = [0.0,0.0,0.0]
    N = [1.0]
    orientaciones = [[0,0,0]]
    
    return shell(l, exp, coef, coord, N, orientaciones)
    
end

build_zero_shell (generic function with 1 method)

In [5]:
g_a = build_shell(0,[13.01, 1.962, 0.4446, 0.122],[0.01968500,0.1379770,0.4781480,0.5012400],[0.0,0.0,0.0])
g_b = build_shell(1,[0.07896],[1.0],[0,0,0])
g_c = build_shell(0,[0.141],[1.0],[0.6068,-0.2383,-0.7169])
g_d = build_shell(0,[0.07896],[1.0],[0,0,0])

shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])

## Shell Pair

In [6]:
struct shell_pair
    
    g_a::shell
    g_b::shell
    
    exp::Vector{Float64}
    coef::Vector{Float64}
    coord::Vector{Vector{Float64}}
    alpha::Vector{Float64}
    
end
    
function build_shell_pair(g_a, g_b)
    
    exp = vec((g_a.exp .+ g_b.exp')')
    coef = vec((g_a.coef .* g_b.coef')')

    coord = []
    for i in eachindex(g_a.exp)
        for j in eachindex(g_b.exp)
            append!(coord,[(g_a.exp[i]*g_a.coord+g_b.exp[j]*g_b.coord)/(g_a.exp[i]+g_b.exp[j])])
        end
    end
            
    alpha = vec(((g_a.exp .* g_b.exp')./(g_a.exp .+ g_b.exp'))')
    
    return shell_pair(g_a, g_b, exp, coef, coord, alpha)

end

build_shell_pair (generic function with 1 method)

In [7]:
pairAB = build_shell_pair(g_a,g_b)
pairCD = build_shell_pair(g_c,g_d)

shell_pair(shell(0, [0.141], [0.163992666204993], [0.6068, -0.2383, -0.7169], [1.0], [[0, 0, 0]]), shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]]), [0.21996], [0.017409643323629098], [[0.38897435897435895, -0.15275641025641026, -0.45955128205128204]], [0.05061538461538462])

## Creador de Shell

In [8]:
function get_set_of_shells(element,coord,basis_name;normalized=true,auxiliar=false)

    key_to_l = Dict("S" => 0, "P" => 1, "D" => 2, "F" => 3, "G" => 4, "H" => 5, "I" => 6)

    element = uppercase(element)
    shells = []
    
    # Basis file
    #filename = joinpath(@__DIR__, "../basis/"*basis_name*".gbs")
    #f = open(filename,"r")
    f = open("basis/"*basis_name*".gbs","r")
    lines = readlines(f)
    close(f)
    
    # Find_atom
    atom_line = 0
    for (i,line) in enumerate(lines)
        splitted_line = split(line)
        if(size(splitted_line)[1]>0)
            if(occursin("****",splitted_line[1]))
                next_line = split(lines[i+1])
                if(size(next_line)[1]>0)        
                    if(next_line[1] == element)
                        atom_line = i+1
                        break
                    end
                end
            end
        end
    end

    # Read shells
    num_of_primitives = 0
    l = 0
    exp = []
    coef = []
    for line in lines[atom_line+1:size(lines)[1]]
        splitted_line = split(line)
        if(size(splitted_line)[1]==3)
            exp = []
            coef = []
            try
                l = parse(Int,splitted_line[1])
            catch
                l = key_to_l[splitted_line[1]]
            end
            num_of_primitives = parse(Int,splitted_line[2])
        elseif(size(splitted_line)[1]==2)
            append!(exp,parse(Float64,splitted_line[1]))           
            append!(coef,parse(Float64,splitted_line[2]))
            if(size(exp)[1] == num_of_primitives)
                if(auxiliar)
                    shell = build_aux_shell(l,exp,coef,coord)
                else
                    shell = build_shell(l,exp,coef,coord,normalized=normalized)
                end
                push!(shells,shell)
            end
        end
        if(occursin("****",splitted_line[1]))
            break
        end
    end
            
    return shells
                        
end

get_set_of_shells (generic function with 1 method)

In [9]:
shells = get_set_of_shells("O",[0.0,0.0,0.0],"aug-cc-pvdz",normalized=true)

9-element Vector{Any}:
 shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013], [0.5697017157663264, 1.0583520999829954, 1.7762810557604662, 2.599411889913172, 3.026845995832634, 2.2223943343899455, 0.6477954310334813, 0.011118696018426194], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013], [-0.2524342625070317, -0.48049109075913626, -0.786299661329412, -1.2541699038887442, -1.4912179216030768, -1.6108317778745953, -0.5497983752719344, 0.788282207951121], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [0.3023], [0.2905619240088511], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(1, [17.7, 3.854, 1.046], [3.243619345562938, 2.5672796523366683, 1.1176693232330894], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
 shell(1, [0.2753], [0.2842482843865529], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
 shell(2, [1.185], [3.8368709884748156], [0.0, 0.0, 0.0], [0.5773502691896257, 

In [10]:
function get_all_shells_from_xyz(molecule,basis_name;normalized=true,auxiliar=false)
    molecule = split(molecule,"\n")    
    natoms = size(molecule)[1]
    shells = []
    for iatom in 1:natoms
        atom = split(molecule[iatom])
        if size(atom)[1] >= 1
            shell = get_set_of_shells(atom[1],[parse(Float64,atom[2]),parse(Float64,atom[3]),parse(Float64,atom[4])],basis_name,normalized=normalized,auxiliar=auxiliar)
            append!(shells,shell)
        end
    end
    return shells        
end

get_all_shells_from_xyz (generic function with 1 method)

In [11]:
molecule = """
O   0.000000  0.000000  0.000000
H   0.277400  0.892900  0.254400
H   0.606800 -0.238300 -0.716900
"""

"O   0.000000  0.000000  0.000000\nH   0.277400  0.892900  0.254400\nH   0.606800 -0.238300 -0.716900\n"

In [12]:
shells = get_all_shells_from_xyz(molecule,"aug-cc-pvdz")

19-element Vector{Any}:
 shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013], [0.5697017157663264, 1.0583520999829954, 1.7762810557604662, 2.599411889913172, 3.026845995832634, 2.2223943343899455, 0.6477954310334813, 0.011118696018426194], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013], [-0.2524342625070317, -0.48049109075913626, -0.786299661329412, -1.2541699038887442, -1.4912179216030768, -1.6108317778745953, -0.5497983752719344, 0.788282207951121], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [0.3023], [0.2905619240088511], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(1, [17.7, 3.854, 1.046], [3.243619345562938, 2.5672796523366683, 1.1176693232330894], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
 shell(1, [0.2753], [0.2842482843865529], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
 shell(2, [1.185], [3.8368709884748156], [0.0, 0.0, 0.0], [0.5773502691896257,

In [13]:
function get_Z_xyz(molecule)
    symbol_to_Z = Dict("H" => 1, "He" => 2, "Li" => 3, "Be" => 4, "B" => 5, "C" => 6, "N" => 7, "O" => 8, "F" => 9, "Ne" => 10) 
    molecule = split(molecule,"\n")    
    natoms = size(molecule)[1]

    Zs = []
    coords = []
    for iatom in 1:natoms
        atom = split(molecule[iatom])
        if size(atom)[1] >= 1
            append!(Zs,symbol_to_Z[atom[1]])
            append!(coords,[[parse(Float64,atom[2]),parse(Float64,atom[3]),parse(Float64,atom[4])]])
        end
    end

    return Zs,coords
    
end

get_Z_xyz (generic function with 1 method)

In [14]:
Zs,coords = get_Z_xyz(molecule)

(Any[8, 1, 1], Any[[0.0, 0.0, 0.0], [0.2774, 0.8929, 0.2544], [0.6068, -0.2383, -0.7169]])

In [15]:
function get_nbf(shells)
    
    nbf = 0
    
    for shell in shells
        nbf += size(shell.orientaciones)[1]
    end
    
    return nbf
    
end

get_nbf (generic function with 1 method)

In [16]:
nbf = get_nbf(shells)

43

## MD

In [17]:
function build_R(lt,alpha,Rpq,F,pairAB,pairCD)
    R = [[[[[[0.0 for lz in 0:lt-n-lx-ly] for ly in 0:lt-n-lx] for lx in 0:lt-n] for n in 0:lt] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
    for ii in eachindex(pairAB.exp)
        for jj in eachindex(pairCD.exp)
            for nini in 0:lt
                R[ii][jj][nini+1][1][1][1] = (-2*alpha[ii][jj])^nini*2*pi^(5/2)/(pairAB.exp[ii]*pairCD.exp[jj]*sqrt(pairAB.exp[ii]+pairCD.exp[jj]))*F[ii][jj][nini+1]
                l = 0
                for n in nini-1:-1:0
                    l += 1
                    for lx in 0:l
                        for ly in 0:l-lx
                            lz = l-lx-ly
                            if(lz>=2)
                                R[ii][jj][n+1][lx+1][ly+1][lz+1] += (lz-1)*R[ii][jj][n+2][lx+1][ly+1][lz-1]
                            end
                            if(lz>=1)
                                R[ii][jj][n+1][lx+1][ly+1][lz+1] += Rpq[ii][jj][3]*R[ii][jj][n+2][lx+1][ly+1][lz]
                            end
                        end
                        ly = l-lx
                        if(ly>=2)
                            R[ii][jj][n+1][lx+1][l-lx+1][1] += (ly-1)*R[ii][jj][n+2][lx+1][l-lx-1][1]
                        end
                        if(ly>=1)
                            R[ii][jj][n+1][lx+1][l-lx+1][1] += Rpq[ii][jj][2]*R[ii][jj][n+2][lx+1][l-lx][1]
                        end
                    end
                    lx = l
                    if(lx>=2)
                        R[ii][jj][n+1][l+1][1][1] += (lx-1)*R[ii][jj][n+2][l-1][1][1]
                    end
                    if(lx>=1)
                        R[ii][jj][n+1][l+1][1][1] += Rpq[ii][jj][1]*R[ii][jj][n+2][l][1][1]                
                    end
                end
            end
        end
    end
             
    return R

end

build_R (generic function with 1 method)

In [18]:
function build_E(l1,l2,pair)
    E = [[[[[0.0 for t in 0:la+lb] for lb in 0:l2] for la in 0:l1] for xyz in 1:3] for ii in eachindex(pair.exp)]

    for ii in eachindex(pair.exp)
        for xyz in 1:3
            E[ii][xyz][1][1][1] = exp(-pair.alpha[ii]*(pair.g_b.coord[xyz]-pair.g_a.coord[xyz])^2)
            for la in 0:l1
                for lb in 0:l2
                    for t in 0:la+lb
                        if(lb>0)
                            if(t>0)               
                                E[ii][xyz][la+1][lb+1][t+1] += 1/(2*pair.exp[ii])*E[ii][xyz][la+1][lb][t]
                            end
                            if(t<=la+lb-1)
                                E[ii][xyz][la+1][lb+1][t+1] += (pair.coord[ii][xyz]-pair.g_b.coord[xyz])*E[ii][xyz][la+1][lb][t+1]
                            end
                            if(t+1<=la+lb-1)
                                E[ii][xyz][la+1][lb+1][t+1] += (t+1)*E[ii][xyz][la+1][lb][t+2]
                            end
                        elseif(la>0)
                            if(t>0)               
                                E[ii][xyz][la+1][lb+1][t+1] += 1/(2*pair.exp[ii])*E[ii][xyz][la][lb+1][t]
                            end
                            if(t<=la-1+lb)
                                E[ii][xyz][la+1][lb+1][t+1] += (pair.coord[ii][xyz]-pair.g_a.coord[xyz])*E[ii][xyz][la][lb+1][t+1]
                            end
                            if(t+1<=la-1+lb)
                                E[ii][xyz][la+1][lb+1][t+1] += (t+1)*E[ii][xyz][la][lb+1][t+2]
                            end
                        end
                    end
                end
            end
        end
    end
                                
    return E
    
end

build_E (generic function with 1 method)

## Overlap

In [19]:
function overlap(pairAB)
    
    g_a = pairAB.g_a
    g_b = pairAB.g_b

    Eab = build_E(g_a.l,g_b.l,pairAB)
    result = zeros(size(g_a.orientaciones)[1],size(g_b.orientaciones)[1])

    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
            integral = 0.0
            for ii in eachindex(pairAB.exp)
                t_z = 0
                t_y = 0
                t_x = 0
                integral += Eab[ii][1][lax+1][lbx+1][t_x+1]*
                            Eab[ii][2][lay+1][lby+1][t_y+1]*
                            Eab[ii][3][laz+1][lbz+1][t_z+1]*
                            (pi/pairAB.exp[ii])^1.5*pairAB.coef[ii]
            end           
            result[a,b] = g_a.N[a]*g_b.N[b]*integral
        end
    end
    
    return result
    
end

overlap (generic function with 1 method)

In [20]:
function get_S(shells)
    
    nbf = get_nbf(shells)
    
    S = zeros(nbf,nbf)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pair = build_shell_pair(g_a,g_b)
            results = overlap(pair)
            S[i:i+size(g_a.orientaciones)[1]-1,j:j+size(g_b.orientaciones)[1]-1] = results
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return S
    
end

get_S (generic function with 1 method)

In [21]:
@btime S = get_S(shells)

  2.040 ms (35211 allocations: 2.40 MiB)


43×43 Matrix{Float64}:
  1.0        -0.214063    0.194384   …   0.0175203     0.0527079
 -0.214063    1.0         0.708607       0.0689136     0.207319
  0.194384    0.708607    1.0            0.100239      0.301558
  0.0         0.0         0.0            0.00896798    0.0269792
  0.0         0.0         0.0            0.243657     -0.0105952
  0.0         0.0         0.0        …  -0.0105952     0.215305
  0.0         0.0         0.0            0.0215346     0.0647845
  0.0         0.0         0.0            0.790122     -0.0254419
  0.0         0.0         0.0           -0.0254419     0.72204
  0.0705864   0.629906    0.664284       0.0717632     0.215892
  0.0         0.0         0.0        …   0.036716     -0.0016037
  0.0         0.0         0.0           -0.0016037     0.0324246
  0.0705864   0.629906    0.664284       0.054209      0.213898
  ⋮                                  ⋱                
 -0.020395   -0.0802209  -0.116686       0.0445754     0.0382745
 -0.0656478  -0.258

# Cinética

In [22]:
function Tij(la,lb,xyz,pairAB)
    
    Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)
    
    S = [[[0.0 for j in 0:lb] for i in 0:la] for ii in eachindex(pairAB.exp)]
    T = [[[0.0 for j in 0:lb] for i in 0:la] for ii in eachindex(pairAB.exp)]
    
    for ii in eachindex(pairAB.exp)
        i = 0
        t = 0
        for j in 0:lb
            S[ii][i+1][j+1] = Eab[ii][xyz][i+1][j+1][t+1]*(pi/pairAB.exp[ii])^0.5            
        end
        for i in 1:la
            for j in max(0,lb-la+i):lb
                S[ii][i+1][j+1] = Eab[ii][xyz][i+1][j+1][t+1]*(pi/pairAB.exp[ii])^0.5
            end
        end
    end

    ii = 0
    for jprim in 1:size(pairAB.g_b.exp)[1], iprim in 1:size(pairAB.g_a.exp)[1]
        ii += 1
        i=0
        j=0
        T[ii][i+1][j+1] = (pairAB.g_a.exp[iprim] - 2*pairAB.g_a.exp[iprim]^2 * ((pairAB.coord[ii][xyz]-pairAB.g_a.coord[xyz])^2 + 1/(2*pairAB.exp[ii]))) * S[ii][i+1][j+1]
        for j in 1:lb
            T[ii][i+1][j+1] = (pairAB.coord[ii][xyz]-pairAB.g_b.coord[xyz])*T[ii][i+1][j+1] + pairAB.g_a.exp[iprim]/pairAB.exp[ii]*2*pairAB.g_b.exp[jprim]*S[ii][i+1][j+1] 
            if(j>1)
                T[ii][i+1][j+1] += (j-1)/(2*pairAB.exp[ii])*T[ii][i+1][j-1] - pairAB.g_a.exp[iprim]/pairAB.exp[ii]*(j)*S[ii][i+1][j-1]
            end
        end
        for i in 1:la
            for j in max(0,lb-la+i):lb
                T[ii][i+1][j+1] = (pairAB.coord[ii][xyz]-pairAB.g_a.coord[xyz])*T[ii][i][j+1] + pairAB.g_b.exp[jprim]/pairAB.exp[ii]*2*pairAB.g_a.exp[iprim]*S[ii][i+1][j+1] 
                if(i>1)
                    T[ii][i+1][j+1] += (i-1)/(2*pairAB.exp[ii])*T[ii][i-1][j+1] - pairAB.g_b.exp[jprim]/pairAB.exp[ii]*(i-1)*S[ii][i-1][j+1]
                end
                if(j>0)
                    T[ii][i+1][j+1] += (j)/(2*pairAB.exp[ii])*T[ii][i][j]# - pairAB.g_b.exp[jprim]/pairAB.exp[ii]*(i-1)*S[ii][i-1][j+1]
                end
            end
        end
    end       
        
    return [T[ii][la+1][lb+1] for ii in eachindex(pairAB.exp)]
        
end

Tij (generic function with 1 method)

In [23]:
function kinetic(pairAB)
    
    g_a = pairAB.g_a
    g_b = pairAB.g_b

    Eab = build_E(g_a.l,g_b.l,pairAB)

    result = zeros((size(g_a.orientaciones)[1],size(g_b.orientaciones)[1]))

    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
            Tx = Tij(lax,lbx,1,pairAB)
            Ty = Tij(lay,lby,2,pairAB)
            Tz = Tij(laz,lbz,3,pairAB)
        
            integral = 0.0
            for ii in eachindex(pairAB.exp)
                tx,ty,tz = 0,0,0
                Sx = Eab[ii][1][lax+1][lbx+1][tx+1]*(pi/pairAB.exp[ii])^0.5
                Sy = Eab[ii][2][lay+1][lby+1][ty+1]*(pi/pairAB.exp[ii])^0.5
                Sz = Eab[ii][3][laz+1][lbz+1][tz+1]*(pi/pairAB.exp[ii])^0.5
                integral += pairAB.coef[ii]*(Tx[ii]*Sy*Sz + Ty[ii]*Sx*Sz + Tz[ii]*Sx*Sy)
            end           
            result[a,b] = g_a.N[a]*g_b.N[b]*integral
        end
    end
    
    return result
    
end

kinetic (generic function with 1 method)

In [24]:
pair = build_shell_pair(shells[1],shells[1])
results = kinetic(pair)

1×1 Matrix{Float64}:
 29.18664172532132

In [25]:
function get_T(shells)
    
    nbf = get_nbf(shells)
    
    T = zeros(nbf,nbf)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pair = build_shell_pair(g_a,g_b)
            results = kinetic(pair)
            T[i:i+size(g_a.orientaciones)[1]-1,j:j+size(g_b.orientaciones)[1]-1] = results
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return T
    
end

get_T (generic function with 1 method)

In [26]:
@btime T = get_T(shells)

  13.783 ms (368439 allocations: 25.05 MiB)


43×43 Matrix{Float64}:
  29.1866     -15.9497      0.167796   …   0.00905594    0.0281567
 -15.9497      10.3611      0.471263       0.0316216     0.0979048
   0.167796     0.471263    0.45345        0.0369178     0.113612
   0.0          0.0         0.0            0.00650627    0.0199626
   0.0          0.0         0.0            0.145181     -0.00783964
   0.0          0.0         0.0        …  -0.0076868     0.124151
   0.0          0.0         0.0            0.0117174     0.0357657
   0.0          0.0         0.0            0.354672     -0.0140457
   0.0          0.0         0.0           -0.0138435     0.317019
  -0.74507      1.01063     0.398359       0.0324269     0.100687
   0.0          0.0         0.0        …   0.0312224    -0.00159497
   0.0          0.0         0.0           -0.00157169    0.0269465
  -0.74507      1.01063     0.398359       0.017382      0.0987037
   ⋮                                   ⋱                
  -0.0133038   -0.0463971  -0.0540723      0.018389

# Potential

In [27]:
function build_R_one(coord,pairAB)
    
    p = pairAB.exp
    Rpc = [pairAB.coord[ii]-coord for ii in eachindex(pairAB.exp)]
    F = [[F_n(n,p[ii]*norm(Rpc[ii])^2) for n in 0:pairAB.g_a.l + pairAB.g_b.l] for ii in eachindex(pairAB.exp)]
    lt = pairAB.g_a.l + pairAB.g_b.l
    R = [[[[[0.0 for lz in 0:lt-n-lx-ly] for ly in 0:lt-n-lx] for lx in 0:lt-n] for n in 0:lt] for ii in eachindex(pairAB.exp)]

    for ii in eachindex(pairAB.exp)
        for nini in 0:lt
            R[ii][nini+1][1][1][1] = (-2*p[ii])^nini*2*pi/p[ii]*F[ii][nini+1]
            l = 0
            for n in nini-1:-1:0
                l += 1
                for lx in 0:l
                    for ly in 0:l-lx
                        lz = l-lx-ly
                        if(lz>=2)
                            R[ii][n+1][lx+1][ly+1][lz+1] += (lz-1)*R[ii][n+2][lx+1][ly+1][lz-1]
                        end
                        if(lz>=1)
                            R[ii][n+1][lx+1][ly+1][lz+1] += Rpc[ii][3]*R[ii][n+2][lx+1][ly+1][lz]
                        end
                    end
                    ly = l-lx
                    if(ly>=2)
                        R[ii][n+1][lx+1][l-lx+1][1] += (ly-1)*R[ii][n+2][lx+1][l-lx-1][1]
                    end
                    if(ly>=1)
                        R[ii][n+1][lx+1][l-lx+1][1] += Rpc[ii][2]*R[ii][n+2][lx+1][l-lx][1]
                    end
                end
                lx = l
                if(lx>=2)
                    R[ii][n+1][l+1][1][1] += (lx-1)*R[ii][n+2][l-1][1][1]
                end
                if(lx>=1)
                    R[ii][n+1][l+1][1][1] += Rpc[ii][1]*R[ii][n+2][l][1][1]                
                end
            end
        end
    end
             
    return R

end

build_R_one (generic function with 1 method)

In [28]:
function potential(pairAB,Z,coord)

    g_a = pairAB.g_a
    g_b = pairAB.g_b
    
    R = build_R_one(coord,pairAB)
    Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)
    
    result = zeros(size(g_a.orientaciones)[1],size(g_b.orientaciones)[1])

    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
            integral = 0.0
            for ii in eachindex(pairAB.exp)
                aux3 = 0
                for t_z in 0:laz+lbz
                    aux2 = 0    
                    for t_y in 0:lay+lby
                        aux1 = 0       
                        for t_x in 0:lax+lbx
                            aux1 += Eab[ii][1][lax+1][lbx+1][t_x+1]*R[ii][1][t_x+1][t_y+1][t_z+1]
                        end
                        aux2 += Eab[ii][2][lay+1][lby+1][t_y+1]*aux1
                    end
                    aux3 += Eab[ii][3][laz+1][lbz+1][t_z+1]*aux2
                end
                integral += aux3*pairAB.coef[ii]
            end           
            result[a,b] = g_a.N[a]*g_b.N[b]*integral
        end
    end
    
    return Z*result
    
end

potential (generic function with 1 method)

In [29]:
function get_V(shells,Zs,coords)
    
    nbf = get_nbf(shells)
    
    V = zeros(nbf,nbf)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pair = build_shell_pair(g_a,g_b)
            for (Z,coord) in zip(Zs,coords)
                results = potential(pair,Z,coord)
                V[i:i+size(g_a.orientaciones)[1]-1,j:j+size(g_b.orientaciones)[1]-1] += results
            end
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return V
    
end

get_V (generic function with 1 method)

In [30]:
@btime V = get_V(shells,Zs,coords)

  7.353 ms (139363 allocations: 9.64 MiB)


43×43 Matrix{Float64}:
  63.1717      -24.0258      6.28042    …  -1.42338     0.564777    1.68501
 -24.0258       21.4724      5.99572       -1.41884     0.635445    1.72175
   6.28042       5.99572     8.49146       -2.02189     0.908231    2.45511
   0.0778471     0.333395    0.277099       1.87139     0.116595    0.296192
   0.0576324     0.246822    0.205144       0.0336933   2.15487    -0.006649
  -0.0407197    -0.17439    -0.144943   …   0.248021   -0.085675    1.82483
   0.0126919     0.155724    0.193216       3.49741     0.148864    0.375653
   0.00939618    0.115287    0.143043       0.075548    3.8278     -0.0562975
  -0.0066388    -0.0814549  -0.101066       0.333052   -0.126186    3.40994
   1.32846       6.30037     5.9278        -1.17873     0.642906    1.83596
   0.00304725    0.0433994   0.0375473  …  -0.0649816   0.337858    0.0209583
  -0.0107728    -0.153427   -0.132738      -0.28447    -0.0108647   0.258984
   1.33544       6.39975     6.01379       -1.53762     0

## 4-C ERIs

In [31]:
function ERI(pairAB,pairCD)

    g_a = pairAB.g_a
    g_b = pairAB.g_b
    g_c = pairCD.g_a
    g_d = pairCD.g_b
    
    lt = pairAB.g_a.l + pairAB.g_b.l + pairCD.g_a.l + pairCD.g_b.l

    # Build boys
    alpha = [[pairAB.exp[ii]*pairCD.exp[jj]/(pairAB.exp[ii]+pairCD.exp[jj]) for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
    Rpq = [[pairAB.coord[ii]-pairCD.coord[jj] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
    F = [[[F_n(n,alpha[ii][jj]*norm(Rpq[ii][jj])^2) for n in 0:lt] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]

    # Build R
    R = build_R(lt,alpha,Rpq,F,pairAB,pairCD)

    # Build E
    Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)
    Ecd = build_E(pairCD.g_a.l,pairCD.g_b.l,pairCD)

    result = zeros((size(g_a.orientaciones)[1],size(g_b.orientaciones)[1],size(g_c.orientaciones)[1],size(g_d.orientaciones)[1]))
    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
            for (c,(lcx,lcy,lcz)) in enumerate(g_c.orientaciones)
                for (d,(ldx,ldy,ldz)) in enumerate(g_d.orientaciones)

                    integral = 0.0
                    for ii in eachindex(pairAB.exp)
                        aux7 = 0
                        for t_z in 0:laz+lbz
                            aux6 = 0    
                            for t_y in 0:lay+lby
                                aux5 = 0        
                                for t_x in 0:lax+lbx
                                    aux4 = 0
                                    for jj in eachindex(pairCD.exp)
                                        aux3 = 0
                                        for u_z in 0:lcz+ldz
                                            aux2 = 0
                                            for u_y in 0:lcy+ldy
                                                aux1 = 0
                                                for u_x in 0:lcx+ldx
                                                    aux1 += (-1)^(u_x+u_y+u_z)*Ecd[jj][1][lcx+1][ldx+1][u_x+1]*R[ii][jj][1][t_x+u_x+1][t_y+u_y+1][t_z+u_z+1]
                                                end
                                                aux2 += Ecd[jj][2][lcy+1][ldy+1][u_y+1]*aux1
                                            end
                                            aux3 += Ecd[jj][3][lcz+1][ldz+1][u_z+1]*aux2
                                        end
                                        aux4 += aux3*pairCD.coef[jj]
                                    end
                                    aux5 += Eab[ii][1][lax+1][lbx+1][t_x+1]*aux4
                                end
                                aux6 += Eab[ii][2][lay+1][lby+1][t_y+1]*aux5
                            end
                            aux7 += Eab[ii][3][laz+1][lbz+1][t_z+1]*aux6
                        end
                        integral += aux7*pairAB.coef[ii]
                    end           
                    result[a,b,c,d] = g_a.N[a]*g_b.N[b]*g_c.N[c]*g_d.N[d]*integral
                end
            end
        end
    end
    
    return result
    
end      

ERI (generic function with 1 method)

In [32]:
@btime ERI(pairAB,pairCD)

  6.983 μs (138 allocations: 10.11 KiB)


1×3×1×1 Array{Float64, 4}:
[:, :, 1, 1] =
 0.008391  -0.00329528  -0.0099135

In [33]:
function get_I4(shells)
    
    nbf = get_nbf(shells)
    
    I = zeros(nbf,nbf,nbf,nbf)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pairAB = build_shell_pair(g_a,g_b)
            k = 1
            for g_c in shells
                l = 1
                for g_d in shells
                    pairCD = build_shell_pair(g_c,g_d)
                    results = ERI(pairAB,pairCD)
                    I[i:i+size(g_a.orientaciones)[1]-1,
                      j:j+size(g_b.orientaciones)[1]-1,
                      k:k+size(g_c.orientaciones)[1]-1,
                      l:l+size(g_d.orientaciones)[1]-1] = results
                    l += size(g_d.orientaciones)[1]
                end
                k += size(g_c.orientaciones)[1]
            end
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return I
    
end

get_I4 (generic function with 1 method)

In [34]:
@btime I = get_I4(shells)

  8.919 s (75245891 allocations: 5.26 GiB)


43×43×43×43 Array{Float64, 4}:
[:, :, 1, 1] =
  4.73827    -1.60036     0.596112    0.0         …   0.0532866    0.160307
 -1.60036     1.81027     0.653067    0.0             0.0630373    0.189641
  0.596112    0.653067    0.86819     0.0             0.0848652    0.255308
  0.0         0.0         0.0         1.44303         0.00821231   0.0247058
  0.0         0.0         0.0         0.0             0.216475    -0.00970238
  0.0         0.0         0.0         0.0         …  -0.00970238   0.190512
  0.0         0.0         0.0         0.475178        0.0114077    0.034319
  0.0         0.0         0.0         0.0             0.379633    -0.0134776
  0.0         0.0         0.0         0.0            -0.0134776    0.343567
  0.143028    0.651499    0.608579    0.0             0.0632424    0.190258
  0.0         0.0         0.0         0.0         …   0.0253843   -0.00112781
  0.0         0.0         0.0         0.0            -0.00112781   0.0223663
  0.143028    0.651499    0.608579 

## 2C ERIs

In [35]:
function normalize_aux_shell(g_a)
    
    g_1 = build_zero_shell()
    
    pairAB = build_shell_pair(g_a,g_1)
    results = ERI(pairAB,pairAB)

    N = diag(results[1:size(g_a.orientaciones)[1],1,1:size(g_a.orientaciones)[1],1])
    N = 1.0 ./ (sqrt.(N))
    
    return N
    
    l = g_a.l
    orientaciones = []
    for lx in l:-1:0
        for ly in l-lx:-1:0
            lz = l-lx-ly
            append!(orientaciones,[[lx,ly,lz]])      
        end
    end        
    
    aux_normalized_shell = shell(g_a.l,g_a.exp,g_a.coef,g_a.coord,N,orientaciones)
    
    return aux_normalized_shell
    
end

function build_aux_shell(l,exp,coef,coord)

    orientaciones = []
    for lx in l:-1:0
        for ly in l-lx:-1:0
            lz = l-lx-ly
            append!(orientaciones,[[lx,ly,lz]])      
        end
    end        

    N = ones(Int((l+1)*(l+2)/2))

    tmp_shell = shell(l, exp, coef, coord, N, orientaciones)

    N = normalize_aux_shell(tmp_shell)    
    
    return shell(l, exp, coef, coord, N, orientaciones)
    
end

build_aux_shell (generic function with 1 method)

In [36]:
shells_aux = get_all_shells_from_xyz(molecule,"aug-cc-pvdz-jkfit",normalized=false,auxiliar=true)

52-element Vector{Any}:
 shell(0, [1517.8667506], [1.0], [0.0, 0.0, 0.0], [1904.7905151311072], [[0, 0, 0]])
 shell(0, [489.67952008], [1.0], [0.0, 0.0, 0.0], [463.12165258630614], [[0, 0, 0]])
 shell(0, [176.72118665], [1.0], [0.0, 0.0, 0.0], [129.54359170558996], [[0, 0, 0]])
 shell(0, [63.79223314], [1.0], [0.0, 0.0, 0.0], [36.24640319610661], [[0, 0, 0]])
 shell(0, [25.36649913], [1.0], [0.0, 0.0, 0.0], [11.445396102474305], [[0, 0, 0]])
 shell(0, [9.91354912], [1.0], [0.0, 0.0, 0.0], [3.536646181981861], [[0, 0, 0]])
 shell(0, [4.46453066], [1.0], [0.0, 0.0, 0.0], [1.304742840640154], [[0, 0, 0]])
 shell(0, [1.80177437], [1.0], [0.0, 0.0, 0.0], [0.41969197615897513], [[0, 0, 0]])
 shell(0, [0.80789711], [1.0], [0.0, 0.0, 0.0], [0.1539927336073522], [[0, 0, 0]])
 shell(0, [0.33864327], [1.0], [0.0, 0.0, 0.0], [0.05193776076506855], [[0, 0, 0]])
 shell(1, [120.16030921], [1.0], [0.0, 0.0, 0.0], [3037.2294228145483, 3037.2294228145483, 3037.2294228145483], [[1, 0, 0], [0, 1, 0], [0, 

In [37]:
function get_I2(shells_aux)
    
    g_1 = build_zero_shell()
    
    nbfaux = get_nbf(shells_aux)
    
    I = zeros(nbfaux,nbfaux)
    i = 1
    for g_a in shells_aux
        pairAB = build_shell_pair(g_a,g_1)
        j = 1
        for g_b in shells_aux
            pairCD = build_shell_pair(g_b,g_1)
            results = ERI(pairAB,pairCD)
            I[i:i+size(g_a.orientaciones)[1]-1,
              j:j+size(g_b.orientaciones)[1]-1] = results[1:size(g_a.orientaciones)[1],1,1:size(g_b.orientaciones)[1],1]
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return I
    
end

get_I2 (generic function with 1 method)

In [38]:
@btime I = get_I2(shells_aux)

  22.715 ms (272516 allocations: 18.34 MiB)


171×171 Matrix{Float64}:
  1.0          0.926763    0.781832   …  0.00899273    0.116605
  0.926763     1.0         0.93961       0.0119237     0.154713
  0.781832     0.93961     1.0           0.0153552     0.199589
  0.627276     0.799166    0.939636      0.0197079     0.257417
  0.504281     0.657862    0.814015      0.0245167     0.323934
  0.400729     0.528131    0.669728   …  0.0300513     0.408937
  0.328861     0.435021    0.556826      0.0344917     0.497296
  0.262346     0.347668    0.447111      0.0369301     0.617285
  0.214748     0.284786    0.366895      0.0334873     0.736842
  0.17282      0.229257    0.295606      0.0223338     0.861202
  0.0          0.0         0.0        …  0.000500447   0.00203821
  0.0          0.0         0.0           0.0110003    -0.000800436
  0.0          0.0         0.0           0.00313062    0.00313298
  ⋮                                   ⋱                ⋮
  0.0283893    0.0375933   0.0482485     0.663561      0.0
  0.156776     0.207

## 3C ERIs

In [39]:
function get_I3(shells,shells_aux)
    
    g_1 = build_zero_shell()
    
    nbf = get_nbf(shells)
    nbfaux = get_nbf(shells_aux)
    
    I = zeros(nbf,nbf,nbfaux)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pairAB = build_shell_pair(g_a,g_b)
            k = 1
            for g_c in shells_aux
                pairCD = build_shell_pair(g_c,g_1)
                results = ERI(pairAB,pairCD)
                I[i:i+size(g_a.orientaciones)[1]-1,
                  j:j+size(g_b.orientaciones)[1]-1,
                  k:k+size(g_c.orientaciones)[1]-1] = results[1:size(g_a.orientaciones)[1],1:size(g_b.orientaciones)[1],1:size(g_c.orientaciones)[1],1]
                k += size(g_c.orientaciones)[1]
            end
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return I
    
end

get_I3 (generic function with 1 method)

In [40]:
@btime I = get_I3(shells,shells_aux)

  444.998 ms (4285489 allocations: 302.37 MiB)


43×43×171 Array{Float64, 3}:
[:, :, 1] =
  1.32905    -0.509215    0.130752   …   0.0116692     0.0351054
 -0.509215    0.428692    0.106704       0.0103827     0.0312353
  0.130752    0.106704    0.157335       0.0153649     0.0462235
  0.0         0.0         0.0            0.0014773     0.0044443
  0.0         0.0         0.0            0.0389323    -0.00174535
  0.0         0.0         0.0        …  -0.00174535    0.0342617
  0.0         0.0         0.0            0.0020464     0.00615639
  0.0         0.0         0.0            0.0680983    -0.00241771
  0.0         0.0         0.0           -0.00241771    0.0616286
  0.0266409   0.116565    0.109305       0.0113566     0.0341651
  0.0         0.0         0.0        …   0.00455327   -0.000202301
  0.0         0.0         0.0           -0.000202301   0.00401192
  0.0266409   0.116565    0.109305       0.00917771    0.0339135
  ⋮                                  ⋱                
 -0.0135838  -0.0120863  -0.0178859      0.00111331  